In [ ]:
# all imports once at the begining

import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from matplotlib.lines import Line2D


# The goal of this analysis is to determine if being a left handed batter makes it more likly for you to get onto the allstar team.

In [ ]:
#Load in needed datasets

allstar = pd.read_csv('baseball/core/AllstarFull.csv')

batting = pd.read_csv('baseball/core/people.csv')


In [ ]:
#Split batting up into left, right and switch and remove unneeded data from the csv files

batting = batting[['playerID', 'bats']]

left = batting[batting.bats == 'L']

right = batting[batting.bats == 'R']

both = batting[batting.bats == 'B']

allstar = allstar[['playerID']]

In [ ]:
# getting % of left handded hitters vs right handed hitters vs switch hitters

leftPercent = len(left)/(len(left)+len(right)+len(both))
rightPercent = len(right)/(len(left)+len(right)+len(both))
bothPercent = len(both)/(len(left)+len(right)+len(both))

# print("Players who hit left handed " + str(leftPercent) + "%")
# print("Players who hit right handed " + str(rightPercent) + "%")
# print("Players who hit with bothhands " + str(bothPercent) + "%")

In [ ]:
# gets left, right and switch hitters that are on a allstar team

lefts = 0
rights = 0
switch = 0


for id in allstar['playerID']:
    lefttest = left[left.playerID == id]
    righttest = right[right.playerID == id]
    switches = both[both.playerID == id]
    
    if not lefttest.empty:
        lefts += 1
        
    if not righttest.empty:
        rights += 1
        
    if not switches.empty:
        switch += 1
    
        
# print(lefts)
# print(rights)
# print(switch)

In [ ]:
# get percentages on the all star team

leftPercentAS = lefts/(lefts+rights+switch)
rightPercentAS = rights/(lefts+rights+switch)
switchPercentAS = switch/(lefts+rights+switch)

# print("Players who hit left handed " + str(leftPercentAS) + "%")
# print("Players who hit right handed " + str(rightPercentAS) + "%")
# print("Players who hit switch handed " + str(switchPercentAS) + "%")

In [ ]:
# GRAPHS

data = {'labels': ['Right Handed', 'Left Handed', 'Switch'],
        'values': [rightPercent, leftPercent, bothPercent]}
df = pd.DataFrame(data)

sns.set_style("whitegrid")
plt.figure(figsize=(6,6))
plt.pie(df['values'], labels=df['labels'], autopct='%1.1f%%')
plt.title('All Baseball ')
plt.show()

data = {'labels': ['Right Handed', 'Left Handed', "Switch"],
        'values': [rightPercentAS, leftPercentAS, switchPercentAS]}
df = pd.DataFrame(data)

sns.set_style("whitegrid")
plt.figure(figsize=(6,6))
plt.pie(df['values'], labels=df['labels'], autopct='%1.1f%%')
plt.title('Allstar Team')
plt.show()

### Looking at the data it seems that players are more likley to get onto an allstar team if they hit left handed or switch

# Do hall of fame players make more money?

In [ ]:
#loading datasets and removing bloatware (bloatdata?)

hall_Fame = pd.read_csv('baseball/core/HallOfFame.csv')

salaries = pd.read_csv('baseball/core/Salaries.csv')

salaries = salaries[['yearID', 'playerID', 'salary']]

hall_Fame = hall_Fame[['playerID']]



# gets the salaries and years of each player in the hall of fame and puts it into its own data frame

hf_years = []
hf_salaries = []



for hf_player in hall_Fame.playerID:
    if not salaries[salaries['playerID'] == hf_player].empty:
        
        # these lines get the salaries and years of all the hall of fame players
        for year in salaries[salaries['playerID'] == hf_player].yearID:
            hf_years.append(year)
        for sal in salaries[salaries['playerID'] == hf_player].salary:
            hf_salaries.append(sal)
            
        # this line removes hall of fame players from the salaries pool
        salaries = salaries.drop(salaries[salaries.playerID == hf_player].index)
            
        
            
            
# these lines make the years and salaries of hall of fame players into a csv
hf_dict = {'yearID': hf_years, 'salaries': hf_salaries}

hf_year_sal = pd.DataFrame(hf_dict)
 

In [ ]:
# take average salaries of both and graph them for each year on a line graph

        
startYear = 1985
endYear = 2012

x = startYear
hallFamePlayers = []
normalPlayers = []
y = 0
while y < 1985:
    hallFamePlayers.append(0)
    normalPlayers.append(0)
    y+=1

while x <= endYear:
    i = 0
    j = 0
    for item in salaries[salaries['yearID'] == x].salary:
        i+=1
        j+=item
        
    normalPlayers.append(j / i)
    x+=1
    
    
x = startYear

while x <= endYear:
    i = 0
    j = 0
    for item in hf_year_sal[hf_year_sal['yearID'] == x].salaries:
        i+=1
        j+=item
        
    hallFamePlayers.append(j / i)
    x+=1




In [ ]:
# GRAPH

plt.plot(hallFamePlayers)
plt.plot(normalPlayers)
plt.xlim(1985, 2012)
plt.ylim(0, 10000000)
plt.ticklabel_format(scilimits=(-5, 8))
plt.ylabel('Income')
plt.xlabel('Year')

plt.legend(["Hall of Fame Players", "Other Players"])


plt.show()

### Looking at the data it is clear that on average baseball players who make it into the hall of fame get paid more than those who don't.

# Analysis: which team makes the most money?
## Will a better team make more money?
## Win/Loss ratio vs. Attendance

In [ ]:
df = pd.read_csv('baseball/core/Teams.csv')
# print(df.columns)

# calculate win ratio, group by teamID, and sort by attendance
df = df[['yearID', 'teamID', 'attendance', 'W', 'L']]
df['win_ratio'] = df['W'] / (df['W'] + df['L'])
df.drop(['W', 'L'], axis=1, inplace=True)
df.groupby('teamID').mean()
df.sort_values(by='attendance', ascending=False, inplace=True)

# drop rows with NaN
df.dropna(inplace=True)
# print(df.head(20))

# Standardize attendance by z-score
df['attendance'] = (df['attendance'] - df['attendance'].mean()) / df['attendance'].std()


In [ ]:
# Do the two variables correlate?
correlation = df['attendance'].corr(df['win_ratio'])
print("correlation: " + str(correlation))

Correlation between attendance and win ratio: 0.2444924229670346 <br>
The low positive correlation suggests that there is a weak relationship between the two variables. <br>
The correlation is calculated using df.corr(), which uses Pearson correlation coefficient.

In [ ]:
# Print the regression summary (p-value, t-statistic, etc.)
X = df['attendance']
y = df['win_ratio']
model = sm.OLS(y, X).fit()
print(model.summary())
print('\nThe p-value is 0.040, which is less than 0.05, so we can reject the null hypothesis.')
print('This means that there is a statistically significant relationship between the two variables.')

In [ ]:
# plot
sns.set(style='darkgrid')
sns.relplot(
            x='attendance', 
            y='win_ratio', 
            size=1, 
            height=10,
            aspect=2,
            legend=False,
            data=df
            )

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Team', markerfacecolor='blue', markersize=10),
]

plt.xlabel('Attendance (z-score)')
plt.ylabel('Win Ratio')
plt.legend(handles=legend_elements, title='Mean team win-ratio \n each year', loc='upper left', bbox_to_anchor=(1, 1), ncol=5)
plt.title('Attendance vs. Win Ratio')




### Conclusion: There is a weak positive correlation between attendance and win ratio.